In [ ]:
!pip install torch
!pip install gensim==3.8.3
!pip install unidecode
!pip install sentencepiece
!pip install transformers

In [1]:
from joblib.parallel import cpu_count
import pandas as pd
import numpy as np
import torch
import pickle
import time
import functools
import operator
from joblib import Parallel,delayed
from functools import partial
import sentencepiece as spm 
import psutil
from tqdm import tqdm
import pickle
import os
import re
from transformers.utils.dummy_pt_objects import CamembertModel
from transformers.utils.dummy_sentencepiece_objects import CamembertTokenizer
from unidecode import unidecode
from bs4 import BeautifulSoup
import gensim
import sys
import gc

from fats import BERTScore, Make_Embedding,TextRank,Random_summary,Lead_3,SMHA_Linear_classifier
from fats import Make_Extractive

os.chdir("/home/jovyan/work")

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
Paragraphes=pickle.load(open('Paragraphes.pickle','rb'))
Paragraphes=functools.reduce(operator.iconcat, Paragraphes, [])

from fats import Make_Embedding
from transformers import CamembertTokenizer,CamembertModel
tok=CamembertTokenizer('MLSUM_tokenizer.model')
camem=CamembertModel.from_pretrained("camembert-base")

nphrase=2
TR=TextRank(tok_path='MLSUM_tokenizer.model',cpu=1)
BS=BERTScore('MLSUM_tokenizer.model',cpu=1)

W2V=gensim.models.Word2Vec.load("W2V_all.model")
TRB=partial(TR.make_resume,type='bert',modele=camem,k=nphrase,get_score_only=True)
TRW=partial(TR.make_resume,type='word2vec',modele=W2V,k=nphrase,get_score_only=True)
BSR=BS.make_score
L3=partial(Lead_3,k=nphrase)
RS=partial(Random_summary,get_index_only=True)

taille=100
start=[]
end=[]

Some weights of the model checkpoint at camembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
TR=TextRank(tok_path='MLSUM_tokenizer.model',cpu=1)
e,d=TR.make_embedding_bert(Paragraphes[6],camem=camem)

In [ ]:
fichiers=[i for i in os.listdir() if 'BSR_sortie' in i]
a=[] if len(fichiers)==0 else pickle.load(open(fichiers[0],'rb'))
for i in tqdm(range(len(a),len(Paragraphes))):
    x=BSR(Paragraphes[i])
    a.append(x)
    del x
    gc.collect()
    if i%10==0:
        pickle.dump(a,open("BSR_sortie.pickle",'wb'))

  0%|          | 4/23508 [00:14<22:39:13,  3.47s/it]

In [ ]:
start.append(time.time())
TRB_sortie=Parallel(2)(delayed(TRB)(i) for i in tqdm(Paragraphes[:taille]))
# TRB_sortie=TRB(Paragraphes[0])
pickle.dump(TRB_sortie,open('test/TRB_sortie.pickle','wb'))
end.append(time.time())
print("TRB :",round((end[-1]-start[-1])/60,2),"minutes")

In [ ]:
TRB_sortie=TRB(Paragraphes[:10])

In [ ]:
TRB_sortie_2=[] #pickle.load(open('TRB_sortie.pickle','rb'))
longueur=[]
for i in range(len(TRB_sortie_2),len(Paragraphes)):
    longueur.append(len(Paragraphes[i]))
    start.append(time.time())
    TRB_sortie_2.append(TRB(Paragraphes[i]))
    end.append(time.time())
    if i%250==0:
        print("Achevé :",round((i/len(Paragraphes))*100,2),"%")
        pickle.dump(TRB_sortie_2,open("TRB_sortie.pickle",'wb'))

Achevé : 0.0 %
Achevé : 1.05 %


In [ ]:
BSR_sortie=pickle.load(open('BSR_sortie.pickle','rb'))
erreur=pickle.load(open('BSR_erreur.pickle','rb'))
BSR_sortie

In [ ]:
BSR_sortie=[]
erreur=[]
for i in range(len(Paragraphes)):
    try:
        x=BSR(Paragraphes[i])
        BSR_sortie.append(x)
    except:
        print(i)
        print("Unexpected error:", sys.exc_info())
        erreur.append(i)
    if i%250==0:
        print("Achevé :",round((i/len(Paragraphes))*100,2),"%")
        pickle.dump(BSR_sortie,open("BSR_sortie.pickle",'wb'))
        pickle.dump(erreur,open("BSR_erreur.pickle",'wb'))
        print("Nombre d'erreurs :",round(len(erreur)/len(Paragraphes)*100,2),"%")

0
Unexpected error: (<class 'ValueError'>, ValueError('only one element tensors can be converted to Python scalars'), <traceback object at 0x7fafd6866f50>)
Achevé : 0.0 %
Nombre d'erreurs : 0.0 %
1
Unexpected error: (<class 'KeyboardInterrupt'>, KeyboardInterrupt(), <traceback object at 0x7fafd5c77d70>)
2
Unexpected error: (<class 'ValueError'>, ValueError('only one element tensors can be converted to Python scalars'), <traceback object at 0x7fafd5c40870>)


In [11]:
BSR(Paragraphes[torch.randint(len(Paragraphes),(1,))])

tensor([0.9443, 0.9410, 0.9669, 0.6170, 0.9655, 0.9092, 0.9209, 0.9330, 0.7975,
        0.9526, 0.9336, 0.8616, 0.7908, 0.9259, 0.9285],
       grad_fn=<DivBackward0>)

In [12]:
BSR_sortie=pickle.load(open('BSR_sortie.pickle','rb'))
len(BSR_sortie)

29

In [6]:
import fats
for i in range(1,6):
    s=time.time()
    resu=TR.make_resume(Paragraphes[:i],
                     cpu=1,
                     type_='TextRankBert',
                     k=2,
                     modele=camem,
                     tok="MLSUM_tokenizer.model",
                     get_score_only=True)
    print((time.time()-s)/60/i)


0.0869313398996989
0.07791940172513326
0.08362084759606254
0.09673363069693247
0.10170523166656494


In [4]:
import fats
resu=fats.Resume(Paragraphes[0:2],
                     DL=False,
                     cpu=1,
                     type_='TextRankBert',k=2,modele=camem,
                     tok="MLSUM_tokenizer.model",get_score_only=True)
resu

[[2, 3], [0, 10]]

In [3]:
rendu=pickle.load(open('TRB_sortie.pickle','rb'))
len(rendu)

2801

In [ ]:
rendu=pickle.load(open('TRB_sortie.pickle','rb'))#[] if 
print(len(rendu))
import fats
pas=1

for i in tqdm(range(len(rendu),int(len(Paragraphes)/(pas+1)))):
    resu=fats.Resume(Paragraphes[(i*2):(i+pas)*2],
                     DL=False,
                     cpu=1,
                     type_='TextRankBert',k=2,modele=camem,
                     tok="MLSUM_tokenizer.model",get_score_only=True)
    rendu.append(resu)
               
    if i%100==0:
        pickle.dump(rendu,open('TRB_sortie.pickle','wb'))

  0%|          | 0/9098 [00:00<?, ?it/s]

2801


  2%|▏         | 198/9098 [38:13<23:15:10,  9.41s/it] 